In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from FoodSeg103_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create scribbles

Load the ground truths as batch

In [42]:
img_nums = [n for n in range(0, 4900, 856)] #[1382] #2750 #1234 #2314
gts = load_food_batch(img_nums, load_images=False)[1]
num_imgs = len(gts)
print(f"Total images: {num_imgs}")

Total images: 6


Check and filter for resolution

In [43]:
resol = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}

# Check for images with resolution over a certain threshold
thresh = 250_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

Resolutions:
   0:   196,608 pixels
 856:    65,536 pixels
1712:   174,592 pixels
2568:   224,256 pixels
3424:    65,536 pixels
4280: 9,585,216 pixels (!)


In [44]:
# Filter out those images
gts = {im_num: gt for im_num, gt in gts.items() if resol[im_num] <= thresh}
num_imgs = len(gts)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Images with resolution under 250,000: 5
   0: 196,608 pixels
 856:  65,536 pixels
1712: 174,592 pixels
2568: 224,256 pixels
3424:  65,536 pixels


Define scribbles parameters

In [55]:
# Which scribbles to create
mode = "all"
class_dist = "balanced"
enforce_max_perc = False
bins = [0.25, 1, 5]
margin = 0.75
scribble_width = 3
# suff = ["TEST"] #["w3"] #standing for width 3
s = "TEST"
rel_scribble_len = 10
scribbles_seed = 1

# Where to save the scribbles
folder_path = "./FoodSeg103_results/data/food_run02"

# What to do
save_res = False
show_res = False
print_steps = True

Loop and create scribbles

In [56]:
percentages = {}
for bin in bins:
    percentages[bin] = []
    for img_num, gt in gts.items():
        print(f"IMG {img_num}: bin {bin}, suff {s}")
        np.random.seed(scribbles_seed)
        scribbles, perc_labelled = create_food_scribble(gt, folder_path, img_num, bin=bin, margin=margin, rel_scribble_len=rel_scribble_len, scribble_width=scribble_width, mode=mode,
                                                        class_dist=class_dist, enforce_max_perc=enforce_max_perc,
                                                        save_res=save_res, suff=s, show_res=show_res, image=None, print_steps=print_steps)
        percentages[bin].append(perc_labelled)
        print(f"Annotation percentage: {perc_labelled:2.3f}%\n")
        print("\n")

IMG 0: bin 0.25, suff TEST

max. perc.: 0.25, margin: 0.75, rel_scribble_len: 10.00, width: 3, mode: all, class_dist: balanced, enforce_max_perc: False, print_steps: True

CLASS 1, max. pixel: 0.255% = 120 pixels
   sk_max_pix: 40.00, sq_size: 44, sq_pix_range: (40, 264)
         Adjusting square size and range to 23 (20, 264)
         Sampling skeleton squares - sk_max_pix_left: 40
         Adjusting square size and range to 13 (10, 264)
         Sampling skeleton squares - sk_max_pix_left: 14
         Adjusting square size and range to 8 (5, 264)
         Sampling skeleton squares - sk_max_pix_left: 14
         Adjusting square size and range to 5 (2, 264)
         Sampling skeleton squares - sk_max_pix_left: 14
      prim_sk_squares pix: 37 = 0.078%
         Adjusting square size and range to 23 (20, 264)
         Sampling skeleton squares - sk_max_pix_left: 40
         Adjusting square size and range to 13 (10, 264)
         Sampling skeleton squares - sk_max_pix_left: 40
      sec

Quick report of labelling percentages:

In [57]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

labelled percentage in bin 0.25: mean = 0.2277, std = 0.0103, range = 0.2167 - 0.2457
labelled percentage in bin 1: mean = 0.9012, std = 0.0170, range = 0.8763 - 0.9277
labelled percentage in bin 5: mean = 4.5341, std = 0.1181, range = 4.3518 - 4.6473


Check percentages of saved scribbles

In [ ]:
perc_dict = {}
for bin in bins:
    bin_perc_list = []
    for img_num in range(0,540,20):
        scrib = np.array(Image.open(f"{folder_path}{img_num:03d}_scribbles_{mode}_{bin_for_file(bin)}_{s}.png"))
        perc_labelled = np.sum(scrib > 0) / scrib.size
        bin_perc_list.append(perc_labelled)
    perc_dict[bin] = bin_perc_list

Show the last created scribble with its own ground truth, and (optionally) all other ground truths

In [ ]:
v = napari.Viewer()
v.add_labels(gt, name="Ground Truth")
v.add_labels(scribbles, name="Scribbles")

In [ ]:
for g in gts:
    v.add_labels(gts[g], name=f"GT {g}")